In [13]:
import os

PATH = os.getcwd() + "/.cache/huggingface"
os.environ["HF_HOME"] = PATH
os.environ["HF_DATASETS_CACHE"] = PATH
os.environ["TORCH_HOME"] = PATH

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
from qdrant_client import QdrantClient
from qdrant_client.models import (
    PointStruct,
    Distance,
    VectorParams,
    SparseVectorParams,
    Modifier,
    Prefetch,
    SparseVector,
    FusionQuery,
    Fusion,
)
import pandas as pd
import math
from tqdm.notebook import tqdm
from BM25 import BM25
from pprint import pprint

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("../models/modernbert-embed-base-tokenizer")
model = AutoModel.from_pretrained("../models/modernbert-embed-base-model")
model = model.to(DEVICE)

bm25 = BM25(
    stopwords_dir=os.path.abspath("./stopwords"), languages=["english", "bengali"]
)

In [3]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )


def dense_embedding(texts: list[str]):
    encoded_queries = tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt"
    )

    with torch.no_grad():
        queries_outputs = model(**encoded_queries.to(DEVICE))

    embeddings = mean_pooling(queries_outputs, encoded_queries["attention_mask"])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings.cpu().numpy()

In [ ]:
COLLECTION_NAME = "product_collection_modern_bert_base"
client = QdrantClient(url="http://localhost:6333", timeout=600)

In [ ]:
client.delete_collection(collection_name=COLLECTION_NAME)
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "dense_vector": VectorParams(size=768, distance=Distance.COSINE)
    },
    sparse_vectors_config={"sparse_vector": SparseVectorParams(modifier=Modifier.IDF)},
)

True

In [6]:
product_info_df = pd.read_csv("./final_5000_products.csv")
product_info_df.head(5)

,title,price,description
0,ZKTeco uFace302 Multi-Biometric T&A Access Con...,28500,ZKTeco uFace302 Multi-Biometric T&A Access Con...
1,HP 682 Black Original Ink Advantage Cartridge,1300,"HP 682 Black Original Ink Advantage Cartridge,..."
2,HP 508A Yellow Original LaserJet Toner (Bundle...,24500,What is the price of HP 508A Yellow Toner in B...
3,Transcend ESD300P 1TB Type-C Portable SSD,10500,"Transcend ESD300P 1TB Type-C Portable SSD,The ..."
4,HP 307A Yellow LaserJet Toner Cartridge,30500,"HP 307A Yellow LaserJet Toner Cartridge,HP 307..."


In [7]:
text_for_embedding_format = "Title: {} | Description: {} | Price: {}"
documents = []

for idx, row in product_info_df.iterrows():
    title = row["title"]
    description = row["description"]
    price = row["price"]
    formatted_document = text_for_embedding_format.format(title, description, price)
    documents.append(formatted_document)

In [8]:
bm25.calculate_avg_doc_len(documents)
print(bm25.avg_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors


302.0428


In [9]:
total_row = product_info_df.shape[0]
batch_size = 1
total_batch = math.ceil(total_row / batch_size)

In [10]:
for start in tqdm(range(0, total_row, batch_size)):
    batch = product_info_df.iloc[start : start + batch_size]

    titles = batch["title"].tolist()
    descriptions = batch["description"].tolist()
    prices = batch["price"].tolist()

    texts_for_embedding = [
        text_for_embedding_format.format(title, description, price)
        for title, description, price in zip(titles, descriptions, prices)
    ]
    dense_vectors = dense_embedding(texts_for_embedding)
    sparse_vectors = bm25.raw_embed(texts_for_embedding)

    points = []
    for idx, (batch_idx, row) in enumerate(batch.iterrows()):
        title = row["title"]
        description = row["description"]
        price = row["price"]

        points.append(
            PointStruct(
                id=batch_idx,
                vector={
                    "dense_vector": dense_vectors[idx],
                    "sparse_vector": sparse_vectors[idx],
                },
                payload={
                    "title": title,
                    "description": description,
                    "price": price,
                },
            )
        )

    operation_info = client.upsert(
        collection_name=COLLECTION_NAME, wait=True, points=points
    )
    print(operation_info, end="\r")

  0%|          | 0/5000 [00:00<?, ?it/s]

/home/atiqur-rahman/venv/lib/python3.12/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)


In [ ]:
def query(query_text: str):
    dense_vector = dense_embedding([query_text])[0]
    sparse_vector = bm25.raw_embed([query_text])[0]

    prefetch = [
        Prefetch(query=dense_vector, using="dense_vector", limit=10),
        Prefetch(query=SparseVector(**sparse_vector), using="sparse_vector", limit=10),
    ]

    results = client.query_points(
        collection_name=COLLECTION_NAME,
        prefetch=prefetch,
        query=FusionQuery(fusion=Fusion.RRF),
        with_payload=True,
        limit=5,
    )

    # results = client.query_points(
    #     collection_name=COLLECTION_NAME,
    #     query=dense_vector,
    #     using="dense_vector",
    #     with_payload=True,
    #     limit=5
    # )

    return [{"score": point.score, "payload": point.payload} for point in results.points]

In [41]:
%%time
query_result = query("small phone")
pprint(query_result)

[{'payload': {'description': 'TCL Onetouch 4021,The , is a Compact Feature '
                             'Phone designed for simplicity and reliability. '
                             'With a compact 1.8-inch TFT display with 128 x '
                             '160 pixels resolution\xa0for clear and '
                             'easy-to-read visuals. Powered by the MediaTek '
                             'MT6261D processor, this Feature\xa0 Phone '
                             'ensures smooth performance for basic tasks. With '
                             '4 MB of RAM and 4 MB of internal storage, '
                             'expandable via microSD up to 32 GB, it provides '
                             'adequate space for essential contacts and '
                             'messages. This TCL onetouch 4021 Compact Feature '
                             'Phone supports dual SIM functionality, allowing '
                             'you to manage two numbers simultaneously, 